In [ ]:
#IMPORTATIONS#

# core importations
import caveclient
from caveclient import CAVEclient
import cloudvolume
from cloudvolume import CloudVolume
import pandas as pd
import numpy as np

# dash importations
import dash
from dash import Dash, dcc, html, Input, Output, State, dash_table

In [ ]:
%%time
#FUNCTIONS#

# defines function to convert nm coordinates into FlyWire-usable
def coordConvert(coords):
    x = coords
    x[0] /= 4
    x[1] /= 4
    x[2] /= 40
    return x

# defines function to convert list of coordinates in [4,4,40] resolution to root id #
def coordsToRoot(coords):

    # sets client #
    client = CAVEclient("flywire_fafb_production")

    # sets cloud volume #
    cv = cloudvolume.CloudVolume("graphene://https://prod.flywire-daf.com/segmentation/1.0/fly_v31", use_https=True)

    # determines resolution of volume #
    res = cv.resolution

    # converts coordinates using volume resolution #
    cv_xyz = [int(coords[0]/(res[0]/4)),int(coords[1]/(res[1]/4)),int(coords[2]/(res[2]/40))]

    # sets point by passing converted coordinates into 'download_point' method #
    point = int(cv.download_point(cv_xyz, size=1))

    # looks up root id for that supervoxel using chunkedgraph #
    root_result = client.chunkedgraph.get_root_id(supervoxel_id=point)

    return root_result

def getNuc(ids, query_type):
    
    #sets client#
    client = CAVEclient("flywire_fafb_production")
    
    #gets current materialization version#
    mat_vers = max(client.materialize.get_versions())
    
    #pulls nucleus table results based on query type#
    if query_type == 'coord':
        nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"pt_root_id": coordsToRoot(ids)},
                                            materialization_version = mat_vers)
    elif query_type == 'nuc':
        nuc_df = client.materialize.query_table('nuclei_v1',
                                                filter_in_dict={"id": nucleus_ids},
                                                materialization_version = mat_vers)
    elif query_type == 'root':
        nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"pt_root_id": ids},
                                            materialization_version = mat_vers)
    
    out_df = pd.Dataframe({'Root ID':[nuc_df['pt_root_id']],
                           'Nucleus ID':[nuc_df['id']],
                           'Nucleus Coordinates':[nuc_df['pt_position']]})
        
    return out_df

# defines function to get presynaptic table entry using root id
def getSyn(root_id):
    
    #sets client#
    client = CAVEclient("flywire_fafb_production")
    
    #gets current materialization version#
    mat_vers = max(client.materialize.get_versions())
    
    #gets pre and post synapse tables using root id#
    pre_syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"pre_pt_root_id":[root_id]})
    post_syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"post_pt_root_id":[root_id]})
    
    #counts total pre and post synapses#
    pre_num = len(pre_syn_df)
    post_num = len(post_syn_df)
    
    #gets lists of pre and post synaptic partners#
    downstream_num = len(pre_syn_df['post_pt_root_id'].unique())
    upstream_num = len(post_syn_df['pre_pt_root_id'].unique())
    
    out_df = pd.DataFrame({'Root ID': [root_id],'Incoming Synapses':[post_num],'Outgoing Synapses':[pre_num],
                           'Upstream Parters':[upstream_num],'Downstream Partners':[downstream_num]})
    
    return out_df

# defines function to get edit data
def getEdits(root_id):
    
    #sets client#
    client = CAVEclient("flywire_fafb_production")
    
    #gets changelog dictionary using root id#
    changes_dict = client.chunkedgraph.get_change_log(root_id)
    
    return changes_dict   

# # defines function to build dataframe using root id list and 'options' (list of keywords based on checkboxes)
# def dfBuilderRoot(root_list,options):
#     root_list = [str(i) for i in root_list]             #ensures root ids are formatted as strings to avoid rounding#
#     df = pd.DataFrame(root_list, columns = ['Root ID']) #creates dataframe out of root_id list#

#     if 'nucs' in options:
#         nuc_df = getNucRoot(root_list)
#         df['Nucleus ID'] = nuc_df['id']
#         df['Nucleus Coordinates'] = nuc_df['pt_position'] #adds nucleus coordinates to df#
# #         df['Nucleus Volume'] = nuc_df['volume'] #TEMPORARILY REMOVED#

#     if 'syns' in options:
#         pre_list = []
#         post_list = []
#         total_list = []
#         for i in root_list:
#             try:
#                 pre_list.append(getPreSyn(i))      #pulls synapse data#
#             except:
#                 pre_list += ['n/a']
#             try:
#                 post_list.append(getPostSyn(i))
#             except:
#                 post_list += ['n/a']
#             try:
#                 total_list.append(int(pre_list[-1] + post_list[-1]))
#             except:
#                 total_list += ['n/a']
                
#         df['Incoming Synapses'] = post_list
#         df['Outgoing Synapses'] = pre_list
#         df['Total Synapses'] = total_list
    
#     if 'edits' in options: #CHANGE THIS TO TABULAR LOG#
#         split_list = []
#         merge_list = []
#         edit_list = []
#         for i in root_list:
#             try:
#                 change_dict = getEdits(i)                         #pulls changelog#
#                 split_list.append(change_dict['n_splits'])        #appends split list#
#                 merge_list.append(change_dict['n_mergers'])       #appends merge list#
#                 edit_list.append(split_list[-1] + merge_list[-1]) #appends edit list#
#             except:
#                 split_list += ['n/a']
#                 merge_list += ['n/a']
#                 edit_list += ['n/a']
#         df['Splits'] = split_list
#         df['Mergers'] = merge_list
#         df['Edits'] = edit_list
           
#     return df

# defines function to build dataframe using root id list and 'options' (list of keywords based on checkboxes)
def dfBuilder(id_list, options):
    
    if all([len(i) == 18 for i in id_list]):
        query_type = 'root'
    elif all([len(i) == 7 for i in id_list]):
        query_type = 'nuc'
    elif len(id_list) % 3 == 0:
        query_type = 'coord'
        
    nuc_df = getNuc(id_list,query_type)

    
    
    
    
    
    
    
    nuc_list = [int(i) for i in nuc_list]               #ensures nuc_list are ints#
    nuc_df = getNucNuc(nuc_list)                        #creates nuc_df using nucleus ids#
    root_list = nuc_df['pt_root_id'].to_list()          #converts pt_root_id column of nuc_df to list of root ids#
    root_list = [str(i) for i in root_list]             #ensures root list are strings to avoid rounding#
    df = pd.DataFrame(root_list, columns = ['Root ID']) #creates dataframe out of root_id list#

    if 'nucs' in options:
        df['Nucleus ID'] = nuc_df['id']
        df['Nucleus Coordinates'] = nuc_df['pt_position']

    if 'syns' in options:
        syn_df = pd.DataFrame()
        for i in root_list:
            try:
                syn_df.append(getSyn(i))
            except:
                pre_list += ['n/a']
                
        df['Incoming Synapses'] = post_list
        df['Outgoing Synapses'] = pre_list
        df['Total Synapses'] = total_list
    
    if 'edits' in options: #CHANGE THIS TO TABULAR LOG#
        split_list = []
        merge_list = []
        edit_list = []
        for i in root_list:
            try:
                change_dict = getEdits(i)                         #pulls changelog#
                split_list.append(change_dict['n_splits'])        #appends split list#
                merge_list.append(change_dict['n_mergers'])       #appends merge list#
                edit_list.append(split_list[-1] + merge_list[-1]) #appends edit list#
            except:
                split_list += ['n/a']
                merge_list += ['n/a']
                edit_list += ['n/a']
        df['Splits'] = split_list
        df['Mergers'] = merge_list
        df['Edits'] = edit_list
           
    return df
   

In [ ]:
# DASH APP #

app = dash.Dash(__name__)

# defines layout of various app elements (submission field, checkboxes, button, output table)#
app.layout = html.Div([
    dcc.Textarea(
        id='message_text',
        value='Choose lookup method from dropdown, input coordinates, select output parameters, and click "Submit" button.\nID queries are limited to 20 entries, coordinate lookups must be done one at a time.\nLookup takes ~2 seconds per entry.',
        style={'width': '800px','resize': 'none'},
        rows=3,
        disabled=True,
    ),
    html.Div(dcc.Input(  #defines input field#
        id='input_field', 
        type='text', 
        placeholder='ID Number',
    )),
    html.Br(
    ),
    dcc.Dropdown(
        id='query_type',
        options=[
            {'label': 'Root ID', 'value': 'root_query'},
            {'label': 'Nucleus ID', 'value': 'nuc_query'},
            {'label': 'Coordinates (batch coordinate input not currently supported)', 'value': 'coord_query'},
        ],
        value='root_query',
        style={'max-width': '500px'},
    ),
    html.Br(
    ),
    html.Div(dcc.Checklist(  #defines data selection checkboxes#
        id='checkboxes',
        options=[
            {'label': 'Nucleus Info', 'value': 'nucs'},
            {'label': 'Synapse Count', 'value': 'syns'},
            {'label': 'Edits', 'value': 'edits'},
        ],
        labelStyle={'display': 'block'},
        value=['nucs','syns','edits'],
    )),
    html.Br(
    ),
    html.Button(  #defines submission button#
        'Submit', 
        id='submit_button', 
        n_clicks=0,
    ),
    html.Br(
    ),
    html.Div(dash_table.DataTable(  #defines output table#
        id='table', 
        fill_width=False,  #sets column width to fit text instead of expanding to container width#
        export_format="csv",
    ))
])

# defines callback that takes root ids and desired data selection on button click and generates table
@app.callback(
    Output('table','columns'),           #defines first output location as the 'columns' aspect of 'table'#
    Output('table', 'data'),             #defines second output location as the 'data' aspect of 'table'#
    Output('message_text','value'),      #defines second output location as the 'data' aspect of 'table'#
    Input('submit_button', 'n_clicks'),  #defines trigger as button press (change in the state of the 'n_clicks' aspect of 'submit_button')# 
    State('query_type', 'value'),        #defines first input state as value of 'query_type'#
    State('input_field', 'value'),       #defines second input state as the value of 'input_field'#
    State('checkboxes','value'),         #defines third input state as the value of 'checkboxes'#
    prevent_initial_call=True,
)
def update_output(n_clicks, query_method, ids, checked):
    id_list = str(ids).split(",")                             #splits 'roots' string into list#
    id_list = [int(x.strip(' ')) for x in id_list]            #strips spaces from root_list entries and converts to integers#
    if len(id_list) <= 20:
        if query_method == 'root_query':                          #creates dataframe for root id queries#
            df = dfBuilderRoot(id_list, checked)
        elif query_method == 'nuc_query':                         #creates dataframe for nucleus id queries#
            df = dfBuilderNuc(id_list, checked)
        elif query_method == 'coord_query':                       #creates dataframe for coordinate query#
            derived_id = [coordsToRoot(id_list)]
            df = dfBuilderRoot(derived_id, checked)
        column_list = [{"name": i, "id": i} for i in df.columns]  #creates column list based on dataframe columns#
        df_dict =  df.to_dict('records')                          #converts df to dictionary#
        message_output = 'Choose lookup method from dropdown, input coordinates, select output parameters, and click "Submit" button.\nID queries are limited to 20 entries, coordinate lookups must be done one at a time.'
        output_list = [column_list,df_dict,message_output]        #combines df_dict and column_list into output list#
        return output_list                                        #returns list of column names and data values#
    else:
        return [0,0,'Please limit each query to a maximum of 20 id numbers.']
        

if __name__ == '__main__':
    app.run_server()

In [ ]:
# BELOW THIS LINE IS TEST CODE#

In [ ]:
#FUNCTION TO CHECK FOR OUTDATED ROOT IDS
def rootCheck(root):
    import cloudvolume
    from cloudvolume import CloudVolume
    from caveclient import CAVEclient

    # sets client #
    client = CAVEclient("flywire_fafb_production")

    # sets cloud volume #
    cv = cloudvolume.CloudVolume("graphene://https://prod.flywire-daf.com/segmentation/1.0/fly_v31", use_https=True)
    
    # gets supervoxels for root id #
    sv_list = cv.get_leaves(root)
    
    print(sv_list)
    
    
    
    
    
    
    
    
    
test_id = 720575940550045042
rootCheck(test_id)    

In [ ]:
#FUNCTION FOR NEUROTRANSMITTER DATA
client = CAVEclient("flywire_fafb_production")
mat_vers = max(client.materialize.get_versions())
pre_syn_df = client.materialize.query_table('synapses_nt_v1', 
                                        filter_in_dict={"pre_pt_root_id":[root_id]})
return len(pre_syn_df)

In [ ]:
%%time
# TABULAR CHANGE LOG
root_ids = [720575940629906060,720575940618617536,720575940618059861,720575940625091594,720575940612700450]
client = CAVEclient("flywire_fafb_production")
df = pd.DataFrame()
tab_log = client.chunkedgraph.get_tabular_change_log(root_ids)

print(tab_log)

In [ ]:
!python -V



# import caveclient

# caveclient.__version__

# client = caveclient.CAVEclient("flywire_fafb_production")
# root_ids = [720575940629906060,720575940618617536,720575940618059861,720575940625091594,720575940612700450]
# tab_log = client.chunkedgraph.get_tabular_change_log(root_ids)

In [ ]:
%%time
#ARCHIVED ITERATIVE CHANGELOG FUNCTION#
root_ids = [720575940629906060,720575940618617536,720575940618059861,720575940625091594,720575940612700450]
client = CAVEclient("flywire_fafb_production")
df = pd.DataFrame()
split_list = []
merge_list = []
edit_list = []
for i in root_ids:
    try:
        change_dict = client.chunkedgraph.get_change_log(i)                         #pulls changelog#
        split_list.append(change_dict['n_splits'])        #appends split list#
        merge_list.append(change_dict['n_mergers'])       #appends merge list#
        edit_list.append(split_list[-1] + merge_list[-1]) #appends edit list#
    except:
        split_list += ['n/a']
        merge_list += ['n/a']
        edit_list += ['n/a']
df['Splits'] = split_list
df['Mergers'] = merge_list
df['Edits'] = edit_list

print(df)

In [ ]:
# ARCHIVED COORDINATE LOOKUP

def coordsToRoot(coords):
    # coordinates must be passed as list in [4,4,40] resolution #
    import cloudvolume
    from cloudvolume import CloudVolume
    from caveclient import CAVEclient

    # sets client #
    client = CAVEclient("flywire_fafb_production")

    # sets cloud volume #
    cv = cloudvolume.CloudVolume("graphene://https://prod.flywire-daf.com/segmentation/1.0/fly_v31", use_https=True)

    # determines resolution of volume #
    res = cv.resolution

    # converts coordinates using volume resolution #
    cv_xyz = [int(coords[0]/(res[0]/4)),int(coords[1]/(res[1]/4)),int(coords[2]/(res[2]/40))]

    # sets point by passing converted coordinates into 'download_point' method #
    point = int(cv.download_point(cv_xyz, size=1))

    # should give result of 82686985730511836 #
    print(point)

    # looks up root id for that supervoxel using chunkedgraph #
    root_result = client.chunkedgraph.get_root_id(supervoxel_id=point)

    # should give result of 82686985730511836 #
    return root_result

coordsToRoot([175196, 61786, 3754])